In [1]:

print("Jupyter fonctionne 🎉")


Jupyter fonctionne 🎉


In [2]:
# =========================================
# Stress Test – Global Risk
# Auteur : Carole
# Environnement : JupyterLab (Anaconda)
# =========================================


In [3]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Python :", sys.version)
print("NumPy :", np.__version__)
print("Pandas :", pd.__version__)


Python : 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
NumPy : 1.26.4
Pandas : 2.2.2


## 1. Introduction

L’objectif de ce notebook est de réaliser un **stress test global de portefeuille**, appliqué à la Banque Alpha, dans un contexte macroéconomique dégradé (scénario de **Stagflation Sévère 2026**).

Ce stress test vise à :
- analyser la **sensibilité du portefeuille** aux chocs macroéconomiques,
- comparer deux secteurs stratégiques (Cyclique : **Construction**, Défensif : **Grande Distribution**),
- mesurer l’impact du stress sur la **Probabilité de Défaut (PD)** et sur la **Perte Attendue (EL)**.

Ce travail s’inscrit dans une logique de **Risk Management / Data Science bancaire**, au sein de la fonction **Global Risk** de la Banque Alpha.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Vérification des versions
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", plt.matplotlib.__version__)
print("Seaborn version:", sns.__version__)


In [ ]:
## 2. Chargement des données
## 3. Exploration du portefeuille (EDA)
## 4. Définition du scénario macroéconomique
## 5. Implémentation du stress
## 6. Calcul des indicateurs clés (PD, EL, UL)
## 7. Visualisation des résultats
## 8. Recommandations stratégiques
